In [157]:
import pandas as pd

df = pd.read_csv("../src/taxipred/data/taxi_trip_pricing.csv")
df.head()

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.0,Low,Clear,3.56,0.80,0.32,53.82,36.2624
1,47.59,Afternoon,Weekday,1.0,High,Clear,NaN,0.62,0.43,40.57,NaN
2,36.87,Evening,Weekend,1.0,High,Clear,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,Weekday,4.0,Low,NaN,3.48,0.51,0.15,116.81,36.4698
4,NaN,Evening,Weekday,3.0,High,Clear,2.93,0.63,0.32,22.64,15.6180


In [158]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Trip_Distance_km       950 non-null    float64
 1   Time_of_Day            950 non-null    object 
 2   Day_of_Week            950 non-null    object 
 3   Passenger_Count        950 non-null    float64
 4   Traffic_Conditions     950 non-null    object 
 5   Weather                950 non-null    object 
 6   Base_Fare              950 non-null    float64
 7   Per_Km_Rate            950 non-null    float64
 8   Per_Minute_Rate        950 non-null    float64
 9   Trip_Duration_Minutes  950 non-null    float64
 10  Trip_Price             951 non-null    float64
dtypes: float64(7), object(4)
memory usage: 86.1+ KB


In [159]:
df.describe().T.drop("count", axis=1)

,mean,std,min,25%,50%,75%,max
Trip_Distance_km,27.070547,19.905300,1.2300,12.63250,25.8300,38.40500,146.067047
Passenger_Count,2.476842,1.102249,1.0000,1.25000,2.0000,3.00000,4.000000
Base_Fare,3.502989,0.870162,2.0100,2.73000,3.5200,4.26000,5.000000
Per_Km_Rate,1.233316,0.429816,0.5000,0.86000,1.2200,1.61000,2.000000
Per_Minute_Rate,0.292916,0.115592,0.1000,0.19000,0.2900,0.39000,0.500000
Trip_Duration_Minutes,62.118116,32.154406,5.0100,35.88250,61.8600,89.05500,119.840000
Trip_Price,56.874773,40.469791,6.1269,33.74265,50.0745,69.09935,332.043689


In [160]:
df.isnull().sum()

Trip_Distance_km         50
Time_of_Day              50
Day_of_Week              50
Passenger_Count          50
Traffic_Conditions       50
Weather                  50
Base_Fare                50
Per_Km_Rate              50
Per_Minute_Rate          50
Trip_Duration_Minutes    50
Trip_Price               49
dtype: int64

In [161]:
df[df["Base_Fare"].isnull()].head()

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
1,47.59,Afternoon,Weekday,1.0,High,Clear,NaN,0.62,0.43,40.57,NaN
25,39.47,Afternoon,Weekday,1.0,Low,Clear,NaN,NaN,0.35,7.59,83.6965
43,45.56,Afternoon,Weekday,NaN,Low,Clear,NaN,0.90,0.50,80.80,85.8840
50,48.51,Evening,Weekday,3.0,High,Rain,NaN,1.00,0.48,110.18,104.1764
92,38.28,Afternoon,Weekday,1.0,Medium,Clear,NaN,1.08,0.39,51.33,63.5611


In [162]:
# check total rows where at least 1 column is null
df[df[["Trip_Distance_km", "Base_Fare", "Per_Km_Rate", "Per_Minute_Rate", "Trip_Duration_Minutes", "Trip_Price"]].isnull().any(axis=1)].shape

(266, 11)

In [163]:
rules = {
    "Trip_Price": lambda r: r["Base_Fare"] + (r["Trip_Distance_km"] * r["Per_Km_Rate"]) + (r["Trip_Duration_Minutes"] * r["Per_Minute_Rate"]),
    
    "Base_Fare": lambda r: r["Trip_Price"] - (r["Trip_Distance_km"] * r["Per_Km_Rate"]) - (r["Trip_Duration_Minutes"] * r["Per_Minute_Rate"]),
    
    "Trip_Distance_km": lambda r: (r["Trip_Price"] - r["Base_Fare"] - (r["Trip_Duration_Minutes"] * r["Per_Minute_Rate"])) / r["Per_Km_Rate"],
    
    "Per_Km_Rate": lambda r: (r["Trip_Price"] - r["Base_Fare"] - (r["Trip_Duration_Minutes"] * r["Per_Minute_Rate"])) / r["Trip_Distance_km"],
    
    "Trip_Duration_Minutes": lambda r: (r["Trip_Price"] - r["Base_Fare"] - (r["Trip_Distance_km"] * r["Per_Km_Rate"])) / r["Per_Minute_Rate"],
    
    "Per_Minute_Rate": lambda r: (r["Trip_Price"] - r["Base_Fare"] - (r["Trip_Distance_km"] * r["Per_Km_Rate"])) / r["Trip_Duration_Minutes"]   
    
}

In [164]:
df_filled = df.copy()

updated = True
while updated:
    updated = False
    for col, func in rules.items():
        mask = df_filled[col].isnull()
        if mask.any():
            before = df_filled.loc[mask, col].copy()
            df_filled.loc[mask, col] = df[mask].apply(func, axis=1)
            
            if not df_filled.loc[mask, col].equals(before):
                updated = True

In [165]:
df_filled.isnull().sum()

Trip_Distance_km          6
Time_of_Day              50
Day_of_Week              50
Passenger_Count          50
Traffic_Conditions       50
Weather                  50
Base_Fare                12
Per_Km_Rate              11
Per_Minute_Rate           5
Trip_Duration_Minutes    15
Trip_Price               17
dtype: int64

In [ ]:
# drop rows where time_of_day, day_of_week ... contain nulls
df_filled = df_filled.dropna(subset=["Time_of_Day", "Day_of_Week", "Traffic_Conditions", "Weather"])

In [167]:
df_filled[df_filled[["Trip_Distance_km", "Base_Fare", "Per_Km_Rate", "Per_Minute_Rate", "Trip_Duration_Minutes", "Trip_Price"]].isnull().any(axis=1)].shape

(28, 11)

In [168]:
df_filled[df_filled[["Trip_Distance_km", "Base_Fare", "Per_Km_Rate", "Per_Minute_Rate", "Trip_Duration_Minutes", "Trip_Price"]].isnull().any(axis=1)].head()

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
1,47.59,Afternoon,Weekday,1.0,High,Clear,NaN,0.62,0.43,40.57,NaN
11,48.53,Night,Weekday,3.0,Low,Clear,4.78,NaN,0.50,79.94,NaN
25,39.47,Afternoon,Weekday,1.0,Low,Clear,NaN,NaN,0.35,7.59,83.6965
109,4.77,Morning,Weekend,2.0,High,Clear,NaN,0.96,0.18,105.91,NaN
173,12.10,Morning,Weekend,NaN,Low,Clear,NaN,0.59,0.31,NaN,19.2643


In [ ]:
df_filled.groupby(
    ["Time_of_Day", "Day_of_Week", "Traffic_Conditions", "Weather"]
)["Base_Fare"].mean()

Time_of_Day  Day_of_Week  Traffic_Conditions  Weather
Afternoon    Weekday      High                Clear      3.248214
                                              Rain       3.683000
                                              Snow       2.870000
                          Low                 Clear      3.560714
                                              Rain       3.180000
                                                           ...   
Night        Weekend      Low                 Clear      3.901429
                                              Rain       3.150000
                          Medium              Clear      3.683750
                                              Rain       3.870000
                                              Snow       2.050000
Name: Base_Fare, Length: 65, dtype: float64

In [170]:
df_filled.groupby(
    ["Time_of_Day", "Day_of_Week", "Traffic_Conditions", "Weather"]
)["Base_Fare"].median()

Time_of_Day  Day_of_Week  Traffic_Conditions  Weather
Afternoon    Weekday      High                Clear      2.995
                                              Rain       3.670
                                              Snow       2.930
                          Low                 Clear      3.500
                                              Rain       3.050
                                                         ...  
Night        Weekend      Low                 Clear      4.330
                                              Rain       3.290
                          Medium              Clear      3.800
                                              Rain       4.410
                                              Snow       2.050
Name: Base_Fare, Length: 65, dtype: float64

In [171]:
df_filled["Base_Fare"] = df_filled["Base_Fare"].fillna(df_filled.groupby(["Time_of_Day", "Day_of_Week", "Traffic_Conditions", "Weather"]
)["Base_Fare"].transform("mean").round(2))

In [174]:
df_filled

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.0,Low,Clear,3.56,0.80,0.32,53.82,36.2624
1,47.59,Afternoon,Weekday,1.0,High,Clear,3.25,0.62,0.43,40.57,NaN
2,36.87,Evening,Weekend,1.0,High,Clear,2.70,1.21,0.15,37.27,52.9032
4,8.64,Evening,Weekday,3.0,High,Clear,2.93,0.63,0.32,22.64,15.6180
5,8.64,Afternoon,Weekend,2.0,Medium,Clear,2.55,1.71,0.48,89.33,60.2028
...,...,...,...,...,...,...,...,...,...,...,...
995,5.49,Afternoon,Weekend,4.0,Medium,Clear,2.39,0.62,0.49,58.39,34.4049
996,45.95,Night,Weekday,4.0,Medium,Clear,3.12,0.61,0.50,61.96,62.1295
997,7.70,Morning,Weekday,3.0,Low,Rain,2.08,1.78,0.32,54.18,33.1236
998,47.56,Morning,Weekday,1.0,Low,Clear,2.67,0.82,0.17,114.94,61.2090
